# Forming Time-, Tick-, Dollar-, &c. Bars from Crypto Data

Python implementations of the methods from Marco Lopez de Prado's book Advances in Financial Machine Learning for forming Open High Low Close bars from underlying tick-level market data.

The most typical candlestick is a time-denominated bar: what was the open, high, low, and close price for a certain unit of time. But we can also aggregate market ticks into bars by some other measure; e.g., the open, high, low, and close price for sets of 100 ticks, or sets of $100, or imbalances between sellers and buyers, or more esoteric concepts. This allows our barred time series to match its temporal curvature to that of the market, with more bars when trading is particularly volatile or high-velocity.

In [2]:
"""
Data Structures, Bars, Information Rates
Using BTC/Nano tick data to replicate exercises from chapter 2 
of Lopez de Prado's Advances in Financial Machine Learning

Data downloaded from some sketchy Australian website, no idea what exchange it's from
Includes:
    Price, Volume, Datetime in Unix timestamp (milliseconds), Buy/Sell 

Exercises:
    1) Form bars. Count their bar counts per week; which is highest? which most stable?
"""

import pandas as pd
import numpy as np
from datetime import datetime

## Load and Format Datasets

In [3]:
btc = pd.read_csv(r"C:\Users\brian\Desktop\BTCUSDS.csv").fillna('bfill')
nano = pd.read_csv(r"C:\Users\brian\Desktop\NANOUSDT.csv").fillna('bfill')

btc.rename(columns={'Unknown': 'Datetime'}, inplace=True)
btc['Datetime'] = btc['Datetime'].apply(lambda x: datetime.utcfromtimestamp(x / 1000)) # convert from millisecond unix timestamp
btc['Index'] = btc.index.to_series() # for convenience during grouping, idk if there's a better solution
btc.loc[btc['BuySell']==0, 'BuySell'] = -1
nano.rename(columns={'Unknown': 'Datetime'}, inplace=True)
nano['Datetime'] = nano['Datetime'].apply(lambda x: datetime.utcfromtimestamp(x / 1000)) # convert from millisecond unix timestamp
nano['Index'] = btc.index.to_series() # for convenience during grouping, idk if there's a better solution
nano.loc[nano['BuySell']==0, 'BuySell'] = -1

btc['VolumeUSD'] = btc['VolumeBTC'].mul(btc['Price'], 1)
btc.rename(columns={'VolumeBTC': 'VolumeAsset'}, inplace=True)
nano['VolumeUSD'] = nano['VolumeNano'].mul(nano['Price'], 1)
nano.rename(columns={'VolumeNano': 'VolumeAsset'}, inplace=True)



## Create Bars Function

In [192]:

def createBars(ticks, bartype='time', n_ticks=50, n_dollars=20000, n_asset=10, ema_period=12):
    # Creates bars from raw tick data
    # Options to create bars by time (default), ticks, volume, dollars, imbalances, and runs
    
    # Parameters:
    ## ticks - the raw tick data; ordered by unix timestamp
    ## bartype - the tick-aggregation algorithm to be used:
    #               'time' - by hour
    #               'tick' - by a fixed number of ticks
    #               'volume' - by a fixed volume of the asset
    #               'dollar' - by a fixed dollar amount traded
    #               'tick_imbalance' - by ticks to reach expected imbalance between buyers and sellers
    #               'volume_imbalance' - by ticks to reach expected imbalance between buyer volume and seller volume
    #               'dollar_imbalance' - by ticks to reach expected imbalance between buyer dollars traded and seller dollars traded
    #               'tick_runs' - by number of buyer vs. seller ticks required to reach expected count, non-contiguous
    #               'volume_runs' - by number of buyer vs. seller volume required to reach expected amount
    #               'dollar_runs'- by numer of buyer vs. seller dollars traded to reach expected amount
    
    # Return an aggregated time series with units, open price, high price, low price, close price,
    # volume USD/asset, buy/sell imbalance, buy/sell run, maybe largest trade USD/asset, maybe eventually other stuff
    
    # Returns:
    ## a dataframe with OLHC data, + additional metrics i.e. runs/imbalances/volumes/etc
    
    ## notes:
    # alternativey, include imbalance/run as additional option? idk
    # like, one core function (monad?) to fold the specified data through an arbitrary partition function and then seperate functions for main/run/etc? idk idk
    # a function to produce some sort of aggregatable like .resample() returns, doing a particular one for each bartype (just normal/imbalance/run), which i can then query in a seperate function with min/max/sum/count/custom/etc
    # let's just type out the damn logic first
    
    # write buyer/seller identification algo too, from price delta, compare to existing
    
    # why does this function suck so bad performance-wise? super slow. vectorizable?
    
    

    # column names for returned bars data
    bars = pd.DataFrame(columns=['FirstDatetime',
                                 'LastDatetime',
                                 'TickCount',
                                 'OpenPrice',
                                 'HighPrice',
                                 'LowPrice',
                                 'ClosePrice',
                                 'VolumeUSD',
                                 'VolumeAsset',
                                 'BuySellImbalance',
                                 'BuySellRun'
                                 ])
    
    
    def time_bars(ticks):
        hourly = ticks.resample('H', on='Datetime')
        bars['FirstDatetime'] = hourly['Datetime'].min()
        bars['LastDatetime'] = hourly['Datetime'].max()
        bars['TickCount'] = hourly.count()['Datetime']
        bars['OpenPrice'] = hourly.first()['Price']
        bars['HighPrice'] = hourly.max()['Price']
        bars['LowPrice'] = hourly.min()['Price']
        bars['ClosePrice'] = hourly.last()['Price']
        bars['VolumeUSD'] = hourly.sum()['VolumeUSD']
        bars['VolumeAsset'] = hourly.sum()['VolumeAsset']
        bars['BuySellImbalance'] = hourly.sum()['BuySell']
        bars['BuySellRun'] = np.maximum(round(hourly.count()['Datetime'].div(2)) + hourly.sum()['BuySell'], 
           hourly.count()['Datetime'] - (round(hourly.count()['Datetime'].div(2)) + hourly.sum()['BuySell'])
           )
        
        return bars
    
    
    
    def tick_bars(ticks):
        data = []
        for i in range(int(np.ceil(len(ticks) / n_ticks))):
            ts = pd.Series()
            try:
                # create chuck of size n_tick, starting at iloc[0] and incrementing up
                bar = ticks.iloc[0 + (n_ticks * i) : 0 + (n_ticks * (i + 1))]
            except:
                # if we have an index error, because we're at the end and len(ticks) % 10 != 0
                bar = ticks.iloc[0 + (n_ticks * i) :]
                
            ts['FirstDatetime'] = bar['Datetime'].min()
            ts['LastDatetime'] = bar['Datetime'].max()
            ts['TickCount'] = bar.count()['Datetime']
            ts['OpenPrice'] = bar.iloc[0]['Price']
            ts['HighPrice'] = bar.max()['Price']
            ts['LowPrice'] = bar.min()['Price']
            ts['ClosePrice'] = bar.iloc[-1]['Price']
            ts['VolumeUSD'] = bar.sum()['VolumeUSD']
            ts['VolumeAsset'] = bar.sum()['VolumeAsset']
            ts['BuySellImbalance'] = bar.sum()['BuySell']
            ts['BuySellRun'] = np.maximum(round(bar['Datetime'].count() / 2) + bar.sum()['BuySell'], 
               bar.count()['Datetime'] - (round(bar['Datetime'].count() / 2) + bar.sum()['BuySell'])
               )
            
            ts.name = i
            data.append(ts)

        b = pd.DataFrame(data, columns=bars.columns)
        return b
    
    
    def dollar_bars(ticks):
        data = [] # to hold the data structure until we turn it into a dataframe and return
        tot_dol = 0 # dollars in tick
        prior_idx = 0 # to keep track of where the bar begins for .iloc indexing of tick
        
        for i in range(len(ticks)):
            ds = pd.Series()
            tot_dol += ticks.iloc[i]['VolumeUSD']
            
            if tot_dol >= n_dollars:
                bar = ticks.iloc[prior_idx:i+1]
                prior_idx = i + 1
                tot_dol = 0
            
                ds['FirstDatetime'] = bar['Datetime'].min()
                ds['LastDatetime'] = bar['Datetime'].max()
                ds['TickCount'] = bar.count()['Datetime']
                ds['OpenPrice'] = bar.iloc[0]['Price']
                ds['HighPrice'] = bar.max()['Price']
                ds['LowPrice'] = bar.min()['Price']
                ds['ClosePrice'] = bar.iloc[-1]['Price']
                ds['VolumeUSD'] = bar.sum()['VolumeUSD']
                ds['VolumeAsset'] = bar.sum()['VolumeAsset']
                ds['BuySellImbalance'] = bar.sum()['BuySell']
                ds['BuySellRun'] = np.maximum(round(bar['Datetime'].count() / 2) + bar.sum()['BuySell'], 
                   bar.count()['Datetime'] - (round(bar['Datetime'].count() / 2) + bar.sum()['BuySell'])
                   )
                ds.name = i
                data.append(ds)
        
        b = pd.DataFrame(data, columns=bars.columns)
        
        return b
    
    
    
    def volume_bars(ticks):
        data = [] # to hold the data structure until we turn it into a dataframe and return
        tot_vol = 0 # dollars in tick
        prior_idx = 0 # to keep track of where the bar begins for .iloc indexing of tick
        
        for i in range(len(ticks)):
            vs = pd.Series()
            tot_vol += ticks.iloc[i]['VolumeAsset']
            
            if tot_vol >= n_asset:
                bar = ticks.iloc[prior_idx:i+1]
                prior_idx = i + 1
                tot_vol = 0
            
                vs['FirstDatetime'] = bar['Datetime'].min()
                vs['LastDatetime'] = bar['Datetime'].max()
                vs['TickCount'] = bar.count()['Datetime']
                vs['OpenPrice'] = bar.iloc[0]['Price']
                vs['HighPrice'] = bar.max()['Price']
                vs['LowPrice'] = bar.min()['Price']
                vs['ClosePrice'] = bar.iloc[-1]['Price']
                vs['VolumeUSD'] = bar.sum()['VolumeUSD']
                vs['VolumeAsset'] = bar.sum()['VolumeAsset']
                vs['BuySellImbalance'] = bar.sum()['BuySell']
                vs['BuySellRun'] = np.maximum(round(bar['Datetime'].count() / 2) + bar.sum()['BuySell'], 
                   bar.count()['Datetime'] - (round(bar['Datetime'].count() / 2) + bar.sum()['BuySell'])
                   )
                vs.name = i
                data.append(vs)
        
        b = pd.DataFrame(data, columns=bars.columns)
        
        return b
    
    
    def tick_imbalance_bars(ticks):
        data = []
        prior_idx = 0
        
        for i in range(len(ticks)):
            tick_imbalance = sum(ticks['BuySell'].iloc[prior_idx:i])
            #expected_ti = data['BuySellImbalance'].emw(span=ema_period)
            print('tick_imbalance: ' + str(tick_imbalance))
            
            # expected value as exponential weighted moving average of prior bar tick count
            # using half of the length of the tick bar to start the default expectation of tick length
            ## under assumption of 50/50 buy/sell -- i think this is wrong? 
            #ev_ticks = pd.Series([data[i].loc['TickCount'] for i in range(len(data))])
            #ev_imb = pd.Series([abs(data[i].loc['BuySellImbalance']) for i in range(len(data))])
        
            if len(data) == 0:
                #diff = ema_period - len(ev_ticks)
                ev_ticks = 5
                ev_imb = 0 #, index=range(diff)).append(ev_imb, ignore_index=True)
            else:
                ev_ticks = pd.Series([data[i]['TickCount'] for i in range(len(data))]).ewm(span=len(data)).mean().iloc[-1]
                ev_imb = pd.Series([data[i]['BuySellImbalance'] for i in range(len(data))]).ewm(span=len(data)).mean().iloc[-1] / ev_ticks
            print('ev_ticks: ' + str(ev_ticks))
            print('ev_imb: ' + str(ev_imb))
            expected_ti = ev_ticks * ev_imb
            print('expected_ti: ' + str(expected_ti))
            
            if abs(tick_imbalance) >= expected_ti:
                bar = ticks.iloc[prior_idx:i+1]
                prior_idx = i + 1
                tis = pd.Series()
                
                tis['FirstDatetime'] = bar['Datetime'].min()
                tis['LastDatetime'] = bar['Datetime'].max()
                tis['TickCount'] = bar.count()['Datetime']
                tis['OpenPrice'] = bar.iloc[0]['Price']
                tis['HighPrice'] = bar.max()['Price']
                tis['LowPrice'] = bar.min()['Price']
                tis['ClosePrice'] = bar.iloc[-1]['Price']
                tis['VolumeUSD'] = bar.sum()['VolumeUSD']
                tis['VolumeAsset'] = bar.sum()['VolumeAsset']
                tis['BuySellImbalance'] = bar.sum()['BuySell']
                tis['BuySellRun'] = np.maximum(round(bar['Datetime'].count() / 2) + bar.sum()['BuySell'], 
                   bar.count()['Datetime'] - (round(bar['Datetime'].count() / 2) + bar.sum()['BuySell'])
                   )
                tis.name = i
                data = data.append(tis)
            print(data)
            
        b = pd.DataFrame(data, columns=bars.columns)
        
        return b
            
    
    
    a = {
        'time':time_bars,
        'tick':tick_bars,
        'dollar':dollar_bars,
        'volume':volume_bars,
        'tick_imbalance':tick_imbalance_bars,
        #'dollar_imbalance':dollar_imbalance_bars,
        #'volume_imbalance':volume_imbalance_bars,
        #'tick_runs':tick_runs,
        #'dollar_runs':dollar_runs,
        #'volume_runs':volume_runs,
    }
        
    return a[bartype](ticks)

In [193]:
tick_imbalance_bars = createBars(btc, 'tick_imbalance')

tick_imbalance: 0
ev_ticks: 5
ev_imb: 0
expected_ti: 0
None
tick_imbalance: 0


TypeError: object of type 'NoneType' has no len()

In [12]:
time_bars

,FirstDatetime,LastDatetime,TickCount,OpenPrice,HighPrice,LowPrice,ClosePrice,VolumeUSD,VolumeAsset,BuySellImbalance,BuySellRun
Datetime,,,,,,,,,,,
2019-02-06 03:00:00,2019-02-06 03:00:16.574,2019-02-06 03:55:51.387,84,3393.83,3500.00,3341.00,3381.00,39070.184380,11.588308,14,56.0
2019-02-06 04:00:00,2019-02-06 04:01:20.766,2019-02-06 04:36:29.357,27,3381.00,3381.00,3366.00,3373.88,7971.022146,2.362603,13,27.0
2019-02-06 05:00:00,2019-02-06 05:04:51.931,2019-02-06 05:50:48.688,6,3373.88,3373.88,3368.91,3373.88,2282.950699,0.676900,-2,5.0
2019-02-06 06:00:00,2019-02-06 06:22:36.386,2019-02-06 06:45:37.115,4,3368.51,3368.81,3368.00,3368.81,3091.985262,0.917973,2,4.0
2019-02-06 07:00:00,2019-02-06 07:03:08.529,2019-02-06 07:52:30.696,18,3368.21,3382.73,3366.78,3382.73,10902.669167,3.231328,12,21.0
2019-02-06 08:00:00,2019-02-06 08:08:42.905,2019-02-06 08:45:55.370,2,3380.39,3380.39,3369.51,3369.51,607.189228,0.179740,0,1.0
2019-02-06 09:00:00,2019-02-06 09:06:49.078,2019-02-06 09:53:07.438,8,3368.61,3376.48,3364.58,3364.58,2451.877662,0.727953,4,8.0
2019-02-06 10:00:00,2019-02-06 10:38:53.532,2019-02-06 10:58:18.199,4,3364.70,3375.18,3362.82,3365.00,379.093196,0.112718,-2,4.0
2019-02-06 11:00:00,2019-02-06 11:09:46.632,2019-02-06 11:48:03.544,3,3372.38,3372.38,3367.27,3367.27,438.174197,0.130118,-3,4.0


In [54]:
tick_bars

,FirstDatetime,LastDatetime,TickCount,OpenPrice,HighPrice,LowPrice,ClosePrice,VolumeUSD,VolumeAsset,BuySellImbalance,BuySellRun
0,2019-02-06 03:00:16.574,2019-02-06 04:25:24.890,100,3393.83,3500.00,3341.00,3372.00,45506.918488,13.495618,24.0,74.0
1,2019-02-06 04:25:28.689,2019-02-06 16:18:57.521,100,3372.00,3382.73,3356.65,3370.36,32518.072341,9.649246,10.0,60.0
2,2019-02-06 16:19:49.657,2019-02-07 14:50:28.740,100,3372.78,3379.85,3357.73,3367.61,26496.099034,7.869342,0.0,50.0
3,2019-02-07 15:10:22.095,2019-02-08 07:19:14.516,100,3367.61,3367.61,3344.05,3362.56,149481.655040,44.609443,42.0,92.0
4,2019-02-08 08:52:23.769,2019-02-08 16:48:31.763,100,3365.86,3579.02,3361.55,3579.02,45639.265804,13.175256,-62.0,112.0
5,2019-02-08 16:48:33.718,2019-02-08 20:53:54.072,100,3577.23,3715.31,3577.23,3631.41,46106.655790,12.623831,-10.0,60.0
6,2019-02-08 20:54:50.121,2019-02-09 03:52:54.046,100,3628.41,3646.89,3597.46,3608.28,46174.947871,12.781003,10.0,60.0
7,2019-02-09 03:53:05.681,2019-02-09 13:58:43.074,100,3604.47,3634.11,3603.66,3627.99,40684.835672,11.240699,-2.0,52.0
8,2019-02-09 13:58:54.705,2019-02-09 14:37:42.452,100,3619.81,3645.75,3617.31,3621.10,45103.048654,12.444084,-4.0,54.0
9,2019-02-09 14:42:24.146,2019-02-10 03:36:15.458,100,3618.89,3630.00,3598.52,3610.30,27064.867765,7.482106,-6.0,56.0


In [24]:
dollar_bars

,FirstDatetime,LastDatetime,TickCount,OpenPrice,HighPrice,LowPrice,ClosePrice,VolumeUSD,VolumeAsset,BuySellImbalance,BuySellRun
8,2019-02-06 03:00:16.574,2019-02-06 03:04:02.831,9,3393.83,3393.83,3341.00,3369.00,10557.930538,3.137333,3.0,7.0
40,2019-02-06 03:04:02.831,2019-02-06 03:10:01.085,32,3369.00,3383.26,3367.92,3368.55,10033.754720,2.977857,6.0,22.0
62,2019-02-06 03:10:01.085,2019-02-06 03:30:02.645,22,3368.25,3500.00,3368.25,3373.31,10090.276307,2.991040,0.0,11.0
87,2019-02-06 03:32:02.089,2019-02-06 04:02:11.765,25,3375.49,3386.53,3374.99,3381.00,10209.635134,3.020798,9.0,21.0
118,2019-02-06 04:05:52.385,2019-02-06 06:22:36.386,31,3372.48,3373.88,3366.00,3368.18,11400.343350,3.381885,9.0,25.0
135,2019-02-06 06:22:37.352,2019-02-06 07:52:27.901,17,3368.00,3382.73,3366.78,3382.73,10237.109014,3.034730,9.0,17.0
175,2019-02-06 07:52:30.309,2019-02-06 14:01:55.243,40,3382.73,3382.73,3358.99,3360.68,10661.582960,3.165946,6.0,26.0
227,2019-02-06 14:01:59.922,2019-02-06 19:02:11.007,52,3360.68,3375.18,3356.65,3371.58,10365.031131,3.076809,-12.0,38.0
254,2019-02-06 19:22:56.720,2019-02-07 00:26:51.996,27,3370.48,3370.48,3357.73,3357.73,10428.818801,3.103720,17.0,31.0
297,2019-02-07 00:35:10.371,2019-02-07 14:50:26.430,43,3360.47,3379.85,3360.47,3367.27,10037.180841,2.975780,-11.0,32.0


In [33]:
volume_bars

,FirstDatetime,LastDatetime,TickCount,OpenPrice,HighPrice,LowPrice,ClosePrice,VolumeUSD,VolumeAsset,BuySellImbalance,BuySellRun
69,2019-02-06 03:00:16.574,2019-02-06 03:39:42.150,70,3393.83,3500.00,3341.00,3381.60,33980.832288,10.082988,12.0,47.0
159,2019-02-06 03:39:43.920,2019-02-06 12:10:41.142,90,3381.60,3386.53,3359.40,3372.68,34120.303021,10.113682,26.0,71.0
291,2019-02-06 12:18:51.156,2019-02-07 14:44:48.398,132,3365.92,3379.85,3356.65,3366.07,33847.132592,10.053456,2.0,68.0
351,2019-02-07 14:46:18.084,2019-02-08 02:29:05.634,60,3366.07,3367.61,3353.03,3354.55,36121.607752,10.762634,0.0,30.0
364,2019-02-08 02:29:05.634,2019-02-08 02:29:14.407,13,3354.55,3354.55,3350.03,3350.03,35624.658737,10.624754,13.0,19.0
375,2019-02-08 02:29:14.407,2019-02-08 02:29:21.525,11,3350.03,3350.03,3349.19,3349.19,36410.526234,10.870334,11.0,17.0
386,2019-02-08 02:29:21.525,2019-02-08 02:29:30.213,11,3347.51,3347.51,3344.50,3344.50,34543.411988,10.319903,11.0,17.0
467,2019-02-08 02:29:31.831,2019-02-08 16:41:40.299,81,3344.50,3483.07,3344.05,3483.07,33914.526402,10.001527,-39.0,80.0
514,2019-02-08 16:41:42.088,2019-02-08 16:52:03.200,47,3483.07,3644.26,3479.20,3644.26,35624.710962,10.006341,-33.0,56.0
618,2019-02-08 16:53:22.878,2019-02-08 23:41:13.510,104,3692.79,3715.31,3597.87,3610.12,37200.118431,10.144018,-8.0,60.0


In [104]:
tick_imbalance_bars

,FirstDatetime,LastDatetime,TickCount,OpenPrice,HighPrice,LowPrice,ClosePrice,VolumeUSD,VolumeAsset,BuySellImbalance,BuySellRun
67,2019-02-06 03:00:16.574,2019-02-06 03:38:33.680,68,3393.83,3500.00,3341.00,3381.54,3.297979e+04,9.786912,12.0,46.0
1444,2019-02-06 03:39:41.844,2019-02-12 23:23:20.452,1377,3380.43,3715.31,3344.05,3587.97,6.059881e+05,172.901448,-85.0,774.0
1445,2019-02-12 23:55:43.505,2019-02-12 23:55:43.505,1,3594.05,3594.05,3594.05,3594.05,1.721909e+01,0.004791,-1.0,2.0
1446,2019-02-13 00:58:35.915,2019-02-13 00:58:35.915,1,3592.37,3592.37,3592.37,3592.37,2.391872e+02,0.066582,1.0,1.0
1447,2019-02-13 02:12:29.114,2019-02-13 02:12:29.114,1,3591.34,3591.34,3591.34,3591.34,6.011544e+01,0.016739,1.0,1.0
1448,2019-02-13 03:52:14.400,2019-02-13 03:52:14.400,1,3594.69,3594.69,3594.69,3594.69,1.491437e+01,0.004149,1.0,1.0
1449,2019-02-13 04:53:23.054,2019-02-13 04:53:23.054,1,3596.52,3596.52,3596.52,3596.52,2.565110e+02,0.071322,-1.0,2.0
1450,2019-02-13 04:53:28.566,2019-02-13 04:53:28.566,1,3595.38,3595.38,3595.38,3595.38,5.009802e+01,0.013934,-1.0,2.0
1451,2019-02-13 05:29:25.207,2019-02-13 05:29:25.207,1,3592.49,3592.49,3592.49,3592.49,1.353327e+02,0.037671,1.0,1.0
1452,2019-02-13 05:52:41.328,2019-02-13 05:52:41.328,1,3597.91,3597.91,3597.91,3597.91,4.995806e+02,0.138853,1.0,1.0


In [34]:
btc

,Index,Price,VolumeAsset,Datetime,BuySell,VolumeUSD
0,0,3393.83,0.002750,2019-02-06 03:00:16.574,-1,9.333032
1,1,3383.83,0.007225,2019-02-06 03:01:56.311,-1,24.448172
2,2,3350.00,0.024417,2019-02-06 03:02:34.331,-1,81.796950
3,3,3366.00,0.975583,2019-02-06 03:02:42.377,1,3283.812378
4,4,3341.00,0.167486,2019-02-06 03:02:46.796,1,559.570726
5,5,3365.00,0.050139,2019-02-06 03:03:08.383,1,168.717735
6,6,3365.00,0.899356,2019-02-06 03:03:15.181,1,3026.332940
7,7,3365.00,0.010377,2019-02-06 03:03:18.089,1,34.918605
8,8,3369.00,1.000000,2019-02-06 03:04:02.831,1,3369.000000
9,9,3369.00,0.593647,2019-02-06 03:04:02.831,1,1999.996743


In [162]:
test = []
len(test)

0

In [85]:
expected_ti = pd.Series(25, range(12)).ewm(span=12).mean()

In [88]:
expected_ti.iloc[-1]

25.0

In [131]:
ticks=btc
sum(ticks.iloc[100:1000]['BuySell'])

-22

In [151]:
pd.Series(0)

0    0
dtype: int64